In [8]:
import json
import os.path

from fastapi import APIRouter
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from pydantic import BaseModel
from app.agents.agent import ActionAgent
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate
from pathlib import Path
from langchain.callbacks.streaming_stdout_final_only import (
    FinalStreamingStdOutCallbackHandler,
)
import pandas as pd
import yfinance as yf
import ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover


load_dotenv()
router = APIRouter()


def read_DB(db_name: str) -> dict:
    with open(f"app/database/{db_name}.json", "r", encoding="utf-8") as f:
        result = json.load(f)
    return result


def backtest_single_thesis(
    thesis: str, company_name: str, investment_duration: int
) -> str:
    llm = ChatOpenAI(
        model_name="gpt-4",
        request_timeout=120,
        temperature=0.3,
        streaming=True,
        callbacks=[FinalStreamingStdOutCallbackHandler()],
        verbose=True,

    )
    action_agent = ActionAgent(llm)
    
    perf_template = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=(
                    "You are aim is to complete given goals. Run python code using python_repl and terminal tool when needed."                    
                )
            ),
            HumanMessagePromptTemplate.from_template(
                "Goal 1) Return python code to get company: {company_name} and  Nifty 50 index (NSEI) closing stock price data from yahoo finance \
                of last {investment_duration} years. Store output in pandas dataframe which can be used in next steps. For downloading data use yfinace \
                like this: yfinance.download('GUJTHEM.BO', period='3y'). Don't use pandas_datareader and don't include any markdown as a part of output. \
                {thesis}. Save code in genai_test.py"
            ),
        ]
    )

    prompt = perf_template.format(
        thesis=thesis,
        company_name=company_name,
        investment_duration=investment_duration,
    )


    investment_thesis = action_agent.run(prompt)
    
    print("OUTPUT FROM AGENT", investment_thesis)

In [9]:
backtest_single_thesis(
    thesis='strategy is based on the idea that when a shorter-term, 50 days SMA crosses above a longer-term, 100 days SMA, it might indicate a potential upward price movement, triggering a buy order. Conversely, when the longer-term, 100 days SMA crosses above the shorter-term, 50 days SMA, it might indicate a potential downward price movement, triggering a sell order',
    company_name= 'Gujarat Themis Biosyn Ltd', investment_duration= 3
) 



> Entering new AgentExecutor chain...
The user wants a python code that fetches the closing stock price data for the company 'Gujarat Themis Biosyn Ltd' and 'Nifty 50 index' from yahoo finance for the last 3 years. The data should be stored in a pandas dataframe. The user also wants to implement a strategy based on 50 days and 100 days SMA. The code should be saved in a file named 'genai_test.py'. I can use the python_repl tool to write and test the python code. Once the code is ready, I can use the terminal tool to write the code into 'genai_test.py'.
Action: python_repl
Action Input: 
```python
import yfinance as yf
import pandas as pd

# Download historical data for required securities
gtbl_data = yf.download('GUJTHEM.BO', period='3y')['Close']
nifty50_data = yf.download('^NSEI', period='3y')['Close']

# Create a dataframe to store the closing price data
df = pd.DataFrame({'GUJTHEM.BO': gtbl_data, '^NSEI': nifty50_data})

# Calculate 50 days and 100 days SMA for both securities
df